In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
basepath = "/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/"

sample = dict(
    bkg=[
        f"{basepath}/Run2_Autumn18/QCD/*/ntuple.root",
        f"{basepath}/Run2_UL/*/TTJets/*/ntuple_training.root"
    ]
)

In [3]:
for sample_name, files in sample.items():
    break

In [4]:
def globlist(filelist):
    for files in filelist:
        for f in fc.glob(files):
            yield f

In [5]:
sample_tree = ObjIter([Tree(f, report=False) for f in tqdm(globlist(files)) ])

17it [00:17,  1.05s/it]


In [6]:
def get_abs_scale(t):

    scale = t.scale 

    abs_scale = np.abs(scale)

    abs_norm = np.sum(scale)/np.sum(abs_scale)

    t.abs_norm = abs_norm
    t.extend(abs_scale=abs_norm*abs_scale)
sample_tree.apply(get_abs_scale, report=True)


100%|██████████| 17/17 [00:01<00:00,  9.58it/s]


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

In [7]:
def get_sample_norm(sample_trees):
    sample_abs_scale = sample_trees.abs_scale.cat
    sample_abs_norm = 1/np.sum(sample_abs_scale)
    
    sample_trees.sample_abs_norm = sample_abs_norm
    for tree in sample_trees:
        tree.extend(sample_abs_scale= sample_abs_norm * tree.abs_scale)

get_sample_norm(sample_tree)

In [8]:
def get_max_sample_scale(sample_trees):
    sample_abs_scale = sample_trees.sample_abs_scale.cat
    max_sample_abs_scale = ak.max(sample_abs_scale)

    sample_trees.max_sample_abs_scale = max_sample_abs_scale

get_max_sample_scale(sample_tree)

In [21]:
sample_info = dict(
    bkg = {
        fc.cleanpath(tree.filelist[0].fname):dict(
            file_abs_norm = tree.file_abs_norm,
            sample_abs_norm = sample_tree.sample_abs_norm,
            max_sample_abs_scale = sample_tree.max_sample_abs_scale
        )
        for tree in sample_tree
    }
)

In [22]:
sample_info

{'bkg': {'sample_abs_norm': 0.04158710256841988,
  'max_sample_abs_scale': 0.0012176249727373406,
  'file_abs_norm': {'/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/Run2_Autumn18/QCD/QCD_HT1000to1500_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root': 0.997369405836067,
   '/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/Run2_Autumn18/QCD/QCD_HT100to200_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root': nan,
   '/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/Run2_Autumn18/QCD/QCD_HT1500to2000_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root': 0.9899462847201966,
   '/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/Run2_Autumn18/QCD/QCD_HT2000toInf_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root': 0.9770693216228145,
   '/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass/Run2_Autumn18/QCD/QCD_HT200to300_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntupl